
## For local analysis

If you prefer to develop your code in a Jupyter notebook, you can download <a href="cases/mini_case_3.ipynb">mini_case_3.ipynb</a>. This notebook file has the same text and code snippets that you see on this webpage. After creating the required code, copy-and-paste it into the code-inputs on this webpage for testing. To complete all exercises below, you will also need data file: <a href="data/airport_routes.json" download="airport_routes.json">airport_routes.json</a>

## What is a network?

The proliferation of social media websites has provided us all with at least some exposure to networks. For example, consider the small network depicted in the figure below:

<div align="center">
  <img src='images/simple_example.png' height="400" />
</div>

In this figure, Person A is friends with Persons B, C, D, E and F while Person G is only friends with Person F. The circles that represent individuals are called _nodes_ or _vertices_. The connections formed between the nodes (i.e., when they are friends) are called _edges_. While this illustration helps us to visualize the relationships between nodes (people) in a network, it is common to describe the same information in matrix form to facilitate computation. In particular, let the first row and column in the matrix denote connections for Person A, the second row and column denote connections for Person B and so on. Each entry in the matrix will contain either a 1 or 0, which corresponds to whether or not the individual in row $i$ is friends with the individual in column $j$. This matrix is called an _adjacency matrix_. 

Check your understanding of how networks can be depicted in matrix form by answering the following quiz questions.


In [1]:
# Question 3.01: How many rows should the adjacency matrix of our example network contain?

In [2]:
# Question 3.02: How many columns should the adjacency matrix of our example network contain?

Now, let's take a look at the first two rows of adjacency matrix for the network in the figure above.

$$
\begin{equation}
\begin{array}{ccc}
 & & \\\\
 & & \begin{array}{ccc} A & B & C  & D & E & F & G \end{array}\\\\
 & \begin{array}{c}  A\\\\  B \end{array} &
  \left(\begin{array}{ccccccc}
  \\; 0 & \\;1 & \\;1 & \\;1 & \\;1 & \\;1 & \\;0 \\\\
  \\; 1 & \\;0 & \\;0 & \\;0 & \\;0 & \\;1 & \\;0 \\\\
  \end{array}\right)
\end{array}
\end{equation}
$$

The first matrix entry represents the connection between Person $A$ and Person $A$. In most networks, we do not consider nodes to be connected to themselves, so we have a $0$ for this entry. The entry in the first row and the second column represents the connection between Person $A$ and Person $B$. Since they are connected, we have a $1$ in this entry. Person $A$ and $B$ are both connected to Person $F$.


In [3]:
# Question 3.03: Use a Numpy matrix to complete the adjacency matrix. Hint:
# Think through how many connections there should be in the matrix. Is that
# consistent with the number printed by `print(X.sum())`?

import numpy as np

X = np.matrix(
    [
        [0, 1, 1, 1, 1, 1, 0],
        [1, 0, 0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1, 0],
    ]
)

print(X)
print(X.sum())
print(type(X))


[[0 1 1 1 1 1 0]
 [1 0 0 0 0 1 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]
14
<class 'numpy.matrix'>


As you can see, adjacency matrices are square matrices (i.e., the number of rows equals the number of columns). The next exercise asks you to write a function that checks whether a matrix is square or not.


In [4]:
# Question 3.04: Write a function that is called `is_square` that accepts a
# matrix and returns a `True` if the matrix has the same number of rows and
# columns and `False` otherwise.

import numpy as np

def is_square(X): 
    return X.shape[0] == X.shape[1]


is_square(np.empty((2, 3))) # should return False
is_square(np.empty((3, 3))) # should return True

True

Remember, entries inform us about whether two nodes are connected. Use this property of adjacency matrices to write a function that determines the neighbors of a particular node.


In [5]:
# Question 3.05: Write a function called `neighbors` that accepts an adjacency
# matrix and a node index and that returns a list of the indices of the
# neighbors for a given node. In other words, for a given row, return a list of
# all the column indices that have a non-zero entry for that row. Use a `for`
# loop to generate the result.

def neighbors(X, n):
    return [i for i in range(X.shape[0]) if X[n, i] != 0]




mx = np.matrix([[0, 1], [1, 0]])
neighbors(mx, 0)  # should return [1]
neighbors(mx, 1)  # should return [0]
nb2 = neighbors(X, 2)  # should return [0]
nb5 = neighbors(X, 5)  # should return [0, 1, 6]
nb2, nb5
type(nb5) # should return `list`

list

In [6]:
mx = np.matrix([[0, 1], [1, 0]])
mx

matrix([[0, 1],
        [1, 0]])

## Walks

To understand the relationship between different nodes in a network, it's natural to think about the sequence of edges that connect two given nodes. For example, consider the ways in which Person G is connected to Person D. In particular, consider this sequence of edges,

$$G \rightarrow F \rightarrow A \rightarrow D$$

A sequence of edges connecting two nodes is called a _walk_. This is not the only walk connecting Person G and Person D, however. Consider the following walk:

$$G \rightarrow F \rightarrow B \rightarrow A \rightarrow D$$

While both walks connect Person G and Person D, the second walk is less direct. One way to compare walks is the _length of the walk_. For example, the first walk mentioned above has a length of 3 while the second path has a length of 4. For very small networks like the one in the example above, calculating the length of the walk appears straightforward. However, when the network becomes larger, we'll need a more systematic way to find walks. This is where the adjacency matrix can help. Denote an adjacency matrix as $A$. Note that $A^1$ tells us the number of ways two nodes connect with walks of length 1. Each entry of $A^2$ tells us the number of ways two nodes connect with walks of length 2 and so on. You will prove this in the following exercises.

<div class="resources">
For more on matrix multiplication watch the following `{r} bs_modal("Matrix multiplication", "matrices", inclMD("./cases/modals/matrices.md"))`
</div>


In [7]:
# Question 3.06: Write a function called `matmult` that accepts any two
# matrices of compatible dimensions and returns the product of those two
# matrices using loops. In subsequent sessions, we'll cover how to use NumPy
# functions to do this for us.

import numpy as np


def matmult(A, B):
    result = np.zeros((A.shape[0], B.shape[1]))
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                result[i, j] += A[i, k] * B[k, j]
    return np.matrix(result)

def col_vector_norm(x):
    # Capture the length of the vector (i.e., number of rows for a column vector)
    vector_len = x.shape[0]
    # Initialize the calcuation with zero
    sum_of_squares = 0
    # Loop over each element of the vector
    for i in range(vector_len):
        sum_of_squares += x[i, 0] ** 2
    # Return the square root of the sum of squares
    return np.sqrt(sum_of_squares)


A = np.matrix([[0, 1, 2], [3, 4, 5]])
B = np.matrix([[0, 1], [2, 3], [4, 5]])
matmult(A, B) # should return matrix([[10, 13], [28, 40]])

C = np.matrix([[0, 1, 2], [3, 4, 5], [5, 6, 8]])
matmult(A, C) # should return matrix([[13, 16, 21], [37, 49, 66]])

matrix([[13., 16., 21.],
        [37., 49., 66.]])

In [8]:
# Question 3.07: Create a function called `matpow` that accepts a square matrix
# and a positive integer as input. The function should return the square matrix
# to the power of the integer (e.g., $A^n$). Use your `maltmult` function from
# the previous step and use loops to repeat the matrix calculations. We will
# cover how to use NumPy to do this in subsequent sessions.

def matpow(X, n):
    Y = X 
    for i in range(n-1):
        Y = matmult(Y, X)
    return Y


A = np.matrix([[0, 1], [2, 3]])
matpow(A, 3) # should return matrix([[6, 11], [22, 39]])
#B = matmult(A, A)
#atmult(A, B) # should return matrix([[6, 11], [22, 39]])

matrix([[ 6., 11.],
        [22., 39.]])

Now, let's use these two functions to create a new function that tells us whether a node is connected within a specific walk distance.


In [9]:
# Question 3.08: Create a function that checks whether a node (let's call it
# the `source` node) is connected to another node (let's call this node the
# `target` node). Your function should accept an adjacency matrix, source node,
# target node, and a maximum length. `is_connected` should return a boolean for
# whether or not the source node and target node are connected within a maximum
# walk length.

def is_connected(A, source, target, max_len):
    for i in range(max_len+1):
        m = matpow(A, i)
        if target in neighbors(m, source):
            return True
    return False

is_connected(X, 1, 2, 2) # should return True
is_connected(X, 1, 2, 1) # should return False
is_connected(X, 0, 1, 1) # should return True
is_connected(X, 5, 6, 1) # should return True

True

## Centrality

In a given network, we might be interested in which nodes might be more "influential" than others. In the simple example we've discussed so far, it seems like Person A would be pretty influential in this network. It seems like Person C and Person D would be equally influential, but what about Person G? We can quantify how influential a specific node is in a network using a measure of centrality.

### Degree Centrality

One way to quantify the centrality of a node in a network would be to count the number of its neighbors normalized by the maximum number of neighbors possible.


In [10]:
# Question 3.09: For a network with 10 nodes, what is the maximum number of neighbors any given node could have? Assume a node will not be connected to itself.

"Degree centrality" is a centrality measure which simply counts the number of neighbors and normalizes it by maximum number of neighbors possible. In the next exercise, we'll write a function to calculate degree centrality.


In [11]:
# Question 3.10: Write a function called `degree_centrality` which accepts an
# adjacency matrix and a node index and returns the degree centrality of that
# node.

def degree_centrality(A, n):
    mx = A.shape[0]-1
    nb = len(neighbors(A, n))
    return nb/mx


degree_centrality(X, 1) # should return 0.333 approximately (i.e., 2/6)
degree_centrality(X, 6) # should return 0.166 approximately (i.e., 1/6)
print(2/6)

0.3333333333333333


### Eigenvector centrality

Degree centrality has an intuitive interpretation, but might not fully capture centrality in other settings. For example, consider the following network.
<div align="center">
  <img src='images/bowtie_example.png' height="400" />
</div>


In [12]:
# Question 3.11: What is the degree centrality of node 0? Write your answer as a decimal and round to 3 decimal places (e.g., 0.123)

In [13]:
# Question 3.12: What is the degree centrality of node 4? Write your answer as a decimal and round to 3 decimal places (e.g., 0.123)

While node 0 and node 4 both have the same degree centrality, it seems natural that node 0 might be more central to the network. In particular, node 0 might seem more central to the network because it is connected to more influential neighbors than node 4. One way to capture this phenomenon is to use a different measure of centrality that also takes into account the centrality of a node's neighbors. For this measure of centrality, let $x_i$ denote the centrality of node $i$. In order to define this measure, we can use a scaled sum of the centrality of its neighbors. For example, if node $i$ is connected to node $j$, then $a_{ij}=1$ in the adjacency matrix, and therefore node $j$'s contribution to node $i$'s centrality is: $a_{ij} x_j$. In order to add up the contributions from all of node $i$'s neighbors we can compute: $\sum_j a_{ij} x_j$. Finally, we can scale this sum by a factor $\frac{1}{\lambda}$ to arrive at:

$$
  x_i = \frac{1}{\lambda} \sum_j a_{ij} x_j
$$
What this says is that node $i$'s centrality is the sum of the centralities of all of node $i$'s neighbors times a scale factor. In other words, if my neighbors have higher centrality, then I too will have higher centrality. What is complicated with this measure is that one cannot compute node $i$'s centrality without first computing its neighbors centrality, and their centralities in turn depend on node $i$'s centrality. Therefore, everything must be computed together by solving the linear system of equations:

$$
 Ax = \lambda x
$$

This equation may look familiar to you, and it is the reason why this measure of centrality is called eigenvector centrality. It is because we are ultimately finding the eigenvectors of A (see (<a href="https://www.khanacademy.org/math/linear-algebra/alternate-bases/eigen-everything/v/linear-algebra-introduction-to-eigenvalues-and-eigenvectors" target = "_blank">Khan Academy on Eigenvectors</a>))

How do we find eigenvectors? We can do so numerically using power iteration (also know as the power method). This consists of the following steps.

- Start with a non-zero random vector, $x_0$.
- Iterate the following recursive formula to arrive at a solution. At the $k$-th step of the loop,
$$
  x_{k+1} = \frac{Ax_k}{\left\lVert Ax_k \right\rVert}
$$
where $\left\lVert \cdot \right\rVert$ denotes the L2-norm. Note that the L2-norm is defined for a vector $y = (y_1, \dots, y_n)$ as follows
$$
  \left\lVert y \right\rVert = \sqrt{y_1^2 + y_2^2 + \dots + y_n^2}
$$

The following exercises will help you work through writing your own power method function.


In [14]:
# Question 3.13: Write a function called `col_vector_norm` which accepts a
# column vector, `x`, and returns the norm of the vector as shown in Equation
# (7) using a loop.

import numpy as np


def col_vector_norm(x):
    # Capture the length of the vector (i.e., number of rows for a column vector)
    vector_len = x.shape[0]
    # Initialize the calcuation with zero
    sum_of_squares = 0
    # Loop over each element of the vector
    for i in range(vector_len):
        sum_of_squares += x[i, 0] ** 2
    # Return the square root of the sum of squares
    return np.sqrt(sum_of_squares)

X = np.matrix([[1], [-2], [2]])
print(X)
col_vector_norm(X)  # Should return 3.0 (i.e,. np.sqrt(1**2 + (-2)**2 + 2**2))

[[ 1]
 [-2]
 [ 2]]


3.0

In [15]:
# Question 3.14: Write a function called `find_eigenvector` that accepts an
# adjacency matrix, `A`, a maximum number of iterations, `max_iter`, and
# tolerance level, `tol`. Your function should return the result of the
# recursive formula (6) until either all the values in the eigenvector converge
# within a tolerance level or the maximum number of iterations has been
# reached. Use your `matmult` function to calculate $Ax_k$ and use
# `col_vector_norm` to calculate $\left\lVert Ax_k\right\rVert$

np.random.seed(495)


def find_eigenvector(A, max_iter, tol):
    # Step 1: Initialize a random vector of the same length as the number of rows in A
    n = A.shape[0]  # Number of rows (or columns, since it's an adjacency matrix)
    x = np.random.rand(n, 1)  # Initial random vector
    
    # Step 2: Normalize the initial vector
    x = x / col_vector_norm(x)
    
    # Step 3: Iterate up to max_iter times
    for i in range(max_iter):
        # Step 4: Multiply A by the vector x to get the next eigenvector approximation
        Ax = matmult(A, x)
        
        # Step 5: Normalize the resulting vector
        Ax_norm = col_vector_norm(Ax)
        x_new = Ax / Ax_norm
        
        # Step 6: Check for convergence (if the change between x_new and x is within tolerance)
        if np.allclose(x_new, x, atol=tol):
            print(f"Converged after {i+1} iterations.")
            return x_new
        
        # Step 7: Update x with the new vector
        x = x_new
    
    # If it reaches max_iter without converging
    print(f"Reached maximum iterations ({max_iter}) without full convergence.")
    return x
   


example_A = np.matrix([[0, 1, 1], [1, 0, 1], [1, 1, 0]])
find_eigenvector(example_A, 100, 1e-3) # Should return a column vector approximately equal to np.matrix([[0.58], [0.58], [0.58]])

Converged after 10 iterations.


matrix([[0.57702775],
        [0.57760872],
        [0.57741418]])

In [16]:
# Question 3.14: Write a function called `find_eigenvector` that accepts an
# adjacency matrix, `A`, a maximum number of iterations, `max_iter`, and
# tolerance level, `tol`. Your function should return the result of the
# recursive formula (6) until either all the values in the eigenvector converge
# within a tolerance level or the maximum number of iterations has been
# reached. Use your `matmult` function to calculate $Ax_k$ and use
# `col_vector_norm` to calculate $\left\lVert Ax_k\right\rVert$

np.random.seed(495)


def find_eigenvector(A, max_iter, tol):
    # Initialize the eigenvector (column vector of ones)
    xk = np.ones((A.shape[0], 1))

    for iteration in range(max_iter):
        # Compute Ax_k using matmult
        Axk = matmult(A, xk)

        # Normalize the result using col_vector_norm
        norm_Axk = col_vector_norm(Axk)
        xk_next = Axk / norm_Axk

        # Check if the change between xk and xk_next is within tolerance
        if np.allclose(xk, xk_next, atol=tol):
            print(f"Converged in {iteration} iterations.")
            return xk_next

        # Update xk for the next iteration
        xk = xk_next

    # If max_iter is reached without convergence
    print(f"Did not converge in {max_iter} iterations.")
    return xk
   


example_A = np.matrix([[0, 1, 1], [1, 0, 1], [1, 1, 0]])
find_eigenvector(example_A, 100, 1e-3) # Should return a column vector approximately equal to np.matrix([[0.58], [0.58], [0.58]])

Converged in 1 iterations.


matrix([[0.57735027],
        [0.57735027],
        [0.57735027]])

In [17]:
# Question 3.15: Compare the centrality measures using an adjacency matrix $X$.
# Calculate the degree centrality for each node using your degree_centrality
# function in a list called `degree_centrality_values`. Store the result from
# applying your `find_eigenvector` function to the matrix $X$ with parameters
# `max_iter=1000` and `tol=1e-5` in a variable called
# `eigenvector_centrality_values`.

np.random.seed(495)

# Adjancency matrix from previous example
X = np.matrix(
    [
        [0, 1, 1, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 0, 0, 0, 1, 1, 0],
        [0, 1, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 1, 0, 0],
        [0, 1, 0, 1, 0, 0, 0],
    ]
)

degree_centrality_values =  [degree_centrality(X, i) for i in range(X.shape[0])]
eigenvector_centrality_values = find_eigenvector(X, 1000, 1e-5)

degree_centrality_values, eigenvector_centrality_values


Converged in 35 iterations.


([0.3333333333333333,
  0.5,
  0.5,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333],
 matrix([[0.38381509],
         [0.4496124 ],
         [0.4496124 ],
         [0.33480717],
         [0.33480717],
         [0.33480717],
         [0.33480717]]))

In [18]:
tm = np.matrix([[0, 1, 1], [1, 0, 1], [5, 5, 5]])
tm.T

matrix([[0, 1, 5],
        [1, 0, 5],
        [1, 1, 5]])

In [19]:
X = np.matrix(
    [
        [0, 1, 1, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 0, 0, 0, 1, 1, 0],
        [0, 1, 0, 0, 0, 0, 1],
        [0, 0, 1, 1, 0, 1, 0],
        [0, 0, 1, 0, 1, 0, 0],
        [0, 1, 0, 1, 0, 0, 0],
    ]
)
print(neighbors(X, 4))  # should return [0, 3, 6]
print(neighbors(X.T, 4))

[2, 3, 5]
[2, 5]


Eigenvector centrality turns out to be a pretty powerful way to assign centrality in a network. In fact, [PageRank](https://en.wikipedia.org/wiki/PageRank), the algorithm that launched Google, is closely related to eigenvector centrality. Larry Page and Sergey Brin developed PageRank to rank the importance of websites by whether other "central" websites linked to them. In the next section, we consider one more attribute of network: whether relationships are reciprocal or not.

## Directed versus undirected

So far, we have talked about relationships that are reciprocal (i.e., friends or not). But relationships can also be one-way. For example, on a Twitter user may follow another user, but may not be followed by that same user. These situations are called _directed_ networks. In terms of the adjacency matrix, an _undirected_ network would be symmetric.


In [20]:
# Question 3.16: Write a function called `is_symmetric` that accepts a square
# matrix as input and returns a boolean equal to `True` when the matrix is
# symmetric and `False` otherwise. Try to do this using a loop.

import numpy as np


def is_symmetric(X):
    if is_square(X) == False:
        return "Matrix is not square"
    for i in range(X.shape[0]):
        if neighbors(X, i) != neighbors(X.T, i):
            return False
    return True


X = np.matrix([[0, 1], [1, 0]])
is_symmetric(X)  # should return True
X = np.matrix([[0, 0], [1, 0]])
is_symmetric(X)  # should return False

False

In [21]:
# Question 3.17: If you used Code Interpreter in ChatGPT+, please provide the link to the chat session you used here. You can find the link by clicking on the 'Share' button in the upper right corner of the ChatGPT+ window as shown in the screenshot below. If you used another AI-tool, please provide your prompt and the URL for the tool.

<div style="text-align:center;">
  <img src="images/ebike-trails.png" height="500px" />
</div>


In [22]:
# Question 3.18: Please write a short reflection on what AI-tools were and were not useful for when you worked on this case. 3-4 lines of text should be sufficient.

## GitHub Extension for Case 3, Part B

Your next step is to complete an extension of the exercise above using Git and GitHub. Click on the link below that corresponds to your section and follow the instructions to complete the extension.

* FW MSBA: <a href="https://classroom.github.com/a/I2K4J7EJ" target="_blank" rel="noopener noreferrer">https://classroom.github.com/a/I2K4J7EJ</a>
* FT MSBA: <a href="https://classroom.github.com/a/ztRBUi16" target="_blank" rel="noopener noreferrer">https://classroom.github.com/a/ztRBUi16</a>

Once you accept the assignment, you will be able to access your repository through the provided link and also at <a href="https://github.com/rsm-msba-24-25" target="_blank" rel="noopener noreferrer">https://github.com/rsm-msba-24-25</a>.

After cloning the repo to the `~/git` directory on your computer, read the README.md file and follow the instructions. Once you push your submission to the server and the code passes all tests you will be done with your work on this platform! Good luck!



